In [1]:
from plyfile import PlyData, PlyElement
from PIL import Image
import numpy as np
import torch
import sys
import os
from pathlib import Path
import pickle

## Read Data into Dataloader

In [2]:
from typing import NamedTuple, Optional, Union
from tqdm import tqdm

In [3]:
class CameraInfo(NamedTuple):
    uid: int
    extr: np.array
    intr: np.array
    image: Optional[np.array]
    frame: int
    width: int = 940
    height: int = 1280
    bg: np.array = np.array([0, 0, 0])
    timestep: Optional[int] = None
    cam_id: Optional[int] = None

In [4]:
with open('/home/yeyiqi/Documents/WD/00122/Outer/Take9/basic_info.pkl', 'rb') as file:
    data = pickle.load(file)
scan_frames = data['scan_frames']
timestep = list(range(len(scan_frames)))
frams_timestep_map = dict(zip(scan_frames, timestep))
frams_timestep_map

{'00011': 0,
 '00012': 1,
 '00013': 2,
 '00014': 3,
 '00015': 4,
 '00016': 5,
 '00017': 6,
 '00018': 7,
 '00019': 8,
 '00020': 9,
 '00021': 10,
 '00022': 11,
 '00023': 12,
 '00024': 13,
 '00025': 14,
 '00026': 15,
 '00027': 16,
 '00028': 17,
 '00029': 18,
 '00030': 19,
 '00031': 20,
 '00032': 21,
 '00033': 22,
 '00034': 23,
 '00035': 24,
 '00036': 25,
 '00037': 26,
 '00038': 27,
 '00039': 28,
 '00040': 29,
 '00041': 30,
 '00042': 31,
 '00043': 32,
 '00044': 33,
 '00045': 34,
 '00046': 35,
 '00047': 36,
 '00048': 37,
 '00049': 38,
 '00050': 39,
 '00051': 40,
 '00052': 41,
 '00053': 42,
 '00054': 43,
 '00055': 44,
 '00056': 45,
 '00057': 46,
 '00058': 47,
 '00059': 48,
 '00060': 49,
 '00061': 50,
 '00062': 51,
 '00063': 52,
 '00064': 53,
 '00065': 54,
 '00066': 55,
 '00067': 56,
 '00068': 57,
 '00069': 58,
 '00070': 59,
 '00071': 60,
 '00072': 61,
 '00073': 62,
 '00074': 63,
 '00075': 64,
 '00076': 65,
 '00077': 66,
 '00078': 67,
 '00079': 68,
 '00080': 69,
 '00081': 70,
 '00082': 71,
 '

In [5]:
with open('/home/yeyiqi/Documents/WD/00122/Outer/Take9/Capture/cameras.pkl', 'rb') as file:
    data = pickle.load(file)
camera_params = data
camera_params

{'0004': {'intrinsics': array([[1.11851456e+03, 3.55271368e-13, 4.55050594e+02],
         [0.00000000e+00, 1.11838198e+03, 6.26902072e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'extrinsics': array([[ 0.99621741, -0.08682953, -0.00339258,  0.25956238],
         [-0.08433436, -0.97552417,  0.20307711,  1.1368941 ],
         [-0.02094263, -0.20202285, -0.97915687,  3.07406384]])},
 '0028': {'intrinsics': array([[1.11937088e+03, 2.13162821e-14, 4.50860062e+02],
         [0.00000000e+00, 1.11941311e+03, 6.33447886e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'extrinsics': array([[ 0.00557261, -0.07967603,  0.99680523,  0.29362592],
         [-0.21553372, -0.9734868 , -0.07660722,  1.119606  ],
         [ 0.9764805 , -0.21441824, -0.02259774,  3.13256862]])},
 '0052': {'intrinsics': array([[1.11749824e+03, 7.10542736e-14, 4.42994563e+02],
         [0.00000000e+00, 1.11751825e+03, 6.31881148e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00

In [6]:
data_path = Path('/home/yeyiqi/Documents/WD/00122/Outer/Take9/Capture')
cam_infos = {}
cam_id = 1
for folder, extr_intr in camera_params.items():
    sys.stdout.write(f"Reading camera {cam_id}/{len(camera_params)}\n")
    sys.stdout.flush()
    folder_path = data_path / folder
    uid = int(folder)
    extr = extr_intr['extrinsics']
    intr = extr_intr['intrinsics']
    frame_idx = 1
    for frame, timestep in frams_timestep_map.items():
        image = np.array(Image.open(folder_path/f'images/capture-f{frame}.png'))
        mask = np.array(Image.open(folder_path/f'masks/mask-f{frame}.png'))
        sys.stdout.write(f"Reading images: {frame_idx}/{len(frams_timestep_map)}\r")
        sys.stdout.flush()
        binary_mask = mask == 255
        result = image.copy()
        result[~binary_mask] = 0
        cam_info = CameraInfo(
            uid = uid,
            extr = extr,
            intr = intr,
            image = result,
            frame = frame,
            timestep = timestep,
            cam_id = cam_id
        )
        if frame not in cam_infos.keys():
            cam_infos[frame] = []
        cam_infos[frame].append(cam_info)
        frame_idx += 1
    sys.stdout.write('\n')
    cam_id += 1
print('Cameras Info Loaded.')

Reading camera 1/4
Reading images: 150/150
Reading camera 2/4
Reading images: 150/150
Reading camera 3/4
Reading images: 150/150
Reading camera 4/4
Reading images: 150/150
Cameras Info Loaded.


In [7]:
class ClothMesh(NamedTuple):
    vertices : np.array
    colors : np.array
    faces : np.array

class SmplxMesh(NamedTuple):
    vertices : np.array
    faces : np.array
    
class SceneInfo(NamedTuple):
    cams_info: list[CameraInfo]
    shoe_mesh: ClothMesh
    upper_mesh:ClothMesh
    lower_mesh: ClothMesh
    outer_mesh: ClothMesh
    smplx_mesh: SmplxMesh

def read_ply_mesh(ply_path, type='cloth'):
    data = PlyData.read(ply_path)
    vertices = np.vstack([data['vertex']['x'], data['vertex']['y'], data['vertex']['z']]).T
    faces = np.vstack(data['face']['vertex_indices'])
    if type == 'cloth':
        colors = np.vstack([
            data['vertex']['red'],
            data['vertex']['green'],
            data['vertex']['blue'],
            data['vertex']['alpha'],
        ]).T
        mesh = ClothMesh(vertices=vertices, colors=colors, faces=faces)
    elif type == 'smplx':
        mesh = SmplxMesh(vertices=vertices, faces=faces)
    else:
        raise TypeError
    return mesh
    
    
def get_mesh_from_frame(frame, cloth_meshes_folder, smplx_mesh_folder):
    cloth_meshes_folder = Path(cloth_meshes_folder)
    smplx_mesh_folder = Path(smplx_mesh_folder)
    shoe_path = cloth_meshes_folder / f'cloth-f{frame}_shoe.ply'
    shoe_mesh = read_ply_mesh(shoe_path)
    upper_path = cloth_meshes_folder / f'cloth-f{frame}_upper.ply'
    upper_mesh = read_ply_mesh(upper_path)
    lower_path = cloth_meshes_folder / f'cloth-f{frame}_lower.ply'
    lower_mesh = read_ply_mesh(lower_path)
    outer_path = cloth_meshes_folder / f'cloth-f{frame}_outer.ply'
    outer_mesh = read_ply_mesh(outer_path)
    smplx_path = smplx_mesh_folder / f'mesh-f{frame}_smplx.ply'
    smplx_mesh = read_ply_mesh(smplx_path, 'smplx')
    return shoe_mesh, upper_mesh, lower_mesh, outer_mesh, smplx_mesh
    

In [8]:
cloth_meshes_folder = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Semantic/clothes'
smplx_mesh_folder = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/SMPLX'
shoe_mesh, upper_mesh, lower_mesh, outer_mesh, smplx_mesh = get_mesh_from_frame('00011', cloth_meshes_folder, smplx_mesh_folder)
f12_Scene_Info = SceneInfo(
    cams_info = cam_infos['00011'],
    shoe_mesh = shoe_mesh,
    upper_mesh = upper_mesh,
    lower_mesh = lower_mesh,
    outer_mesh = outer_mesh,
    smplx_mesh = smplx_mesh
)
f12_Scene_Info

SceneInfo(cams_info=[CameraInfo(uid=4, extr=array([[ 0.99621741, -0.08682953, -0.00339258,  0.25956238],
       [-0.08433436, -0.97552417,  0.20307711,  1.1368941 ],
       [-0.02094263, -0.20202285, -0.97915687,  3.07406384]]), intr=array([[1.11851456e+03, 3.55271368e-13, 4.55050594e+02],
       [0.00000000e+00, 1.11838198e+03, 6.26902072e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), image=array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
 